<a href="https://colab.research.google.com/github/Pansyhb/Git-6300/blob/updates/Copy_of_Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Creation and Training

In [ ]:
!pip install yfinance
!pip install findspark
!pip install pyspark
!pip install vaderSentiment
import pandas as pd
import csv
import datetime
import yfinance as yf
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup as bs
import requests
from pyspark.sql.functions import sum,max,min,mean,count
import datetime as dt
import pyspark
from pyspark.sql import SparkSession
import findspark
import yaml
from yaml.loader import SafeLoader
from os.path import abspath

##Read in CSV Files Containing Data for Target Companies

In [ ]:
from os import listdir
import pandas as pd
path = '/content/Git-6300/data/'

from sklearn.preprocessing import MinMaxScaler
#Function to read in csv files and scale columns for data where scaling of predictions is eventually needed
def scale(path):
    scaler_list = []
    target_list = []
    feat_list = []
    for item in listdir(path):
        print(str(item))
        df = pd.read_csv(path + str(item)).set_index('date')
        print(df)
        #create scaler instances
        scaler = MinMaxScaler(feature_range=(0,1))
        target_scaler = MinMaxScaler(feature_range = (0,1))
        data = df.drop(['ticker'],axis=1)
        # scale features and target columns
        target = target_scaler.fit_transform(data[['target']])
        target = target.flatten()
        scaler_list.append(target_scaler)
        target_list.append(target)
        X_feat = data.drop(['target'], axis = 1)
        for col in X_feat.columns:
            X_feat[col] = scaler.fit_transform(X_feat[[col]])
        feat_list.append(X_feat)
    return scaler_list, target_list, feat_list
scaler_list, target_list, feat_list = scale(path) 

In [ ]:
#clone git to workspace
!git clone https://github.com/Pansyhb/Git-6300.git

Cloning into 'Git-6300'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 35 (delta 14), reused 23 (delta 12), pack-reused 0
Unpacking objects: 100% (35/35), 26.64 KiB | 1.78 MiB/s, done.


In [ ]:
#Split training and test data (microsoft and google) was well as features and targets
test_data = feat_list[0]   
test_target = target_list[0]
train_data = feat_list[1] 
train_target = target_list[1]

In [ ]:
# Split data into X_train and y_train data sets
import numpy as np
def lstm_split(data,target,steps):
      X = []
      y = []
      # Creating a data structure with 10 time-steps and 1 output
      for i in range(10, steps):
          X.append(data[i-10:i])
          y.append(target[i:i+1])  
      return np.array(X),np.array(y)
 
X1,y1 = lstm_split(train_data, train_target,len(train_data))

#Define Training and Test Datasets
train_split = 0.9
split_idx = int(np.ceil(len(X1)*train_split))
date_index = train_data.index
 
X_train,X_test = X1[:split_idx],X1[split_idx:]
y_train,y_test = y1[:split_idx],y1[split_idx:]
X_train_date,X_test_date = date_index[:split_idx],date_index[split_idx:]
 
print(X1.shape,X_train.shape,X_test.shape,y_test.shape,y_train.shape)

(1792, 10, 18) (1613, 10, 18) (179, 10, 18) (179, 1) (1613, 1)


#Configure the base LSTM model 
This process involved running a random search function to select best hyperparameters for the network. The results of this search are availble below.


In [ ]:
!pip install tensorflow
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.metrics import Precision
from keras.optimizers import Adam
!pip install keras_tuner
import keras_tuner

#define function to create model, optional hyperparameters included to be selected during training
LR = 0.05
def build_model(hp):
  model = Sequential()
  hidden = hp.Choice('n_hidden', [0,1,2,3])
  model.add(LSTM(units = hp.Int('neurons_visible', min_value = X_train.shape[2], max_value = 100, step = 20),
                activation = hp.Choice('activate1', ['sigmoid', 'relu']),
                input_shape = (X_train.shape[1], X_train.shape[2]),
                return_sequences = True if hidden >0 else False))
  #Configure hidden layers based on random search determined hidden layer number
  if hidden > 0:
    for num in range(hidden):
      model.add(Dropout(hp.Float('dropout' +str(num+1), min_value = 0.1, max_value = 0.9, step = 0.3)))
      model.add(LSTM(units = hp.Int('neurons_hidden'+str(num+1), min_value = 20, max_value = 50, step = 10),
                     activation = 'relu', return_sequences = True if num != hidden else False))
      
  model.add(Dropout(0.5))
  model.add(Dense(units = 1, activation = 'sigmoid'))

  #compile the model
  model.compile(loss = 'mean_squared_error', optimizer = Adam(lr = LR), metrics = ['mean_squared_error'])
  
  return model

#set learning rate and early stopping callbacks
LR_decay = ReduceLROnPlateau('loss', patience=1, verbose=0, 
                             factor=0.5, min_lr=1e-8)
Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                           patience=25, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)
#arrange random search class
tune = keras_tuner.RandomSearch(build_model, objective = 'val_loss', max_trials = 50, seed = 1)

#complete training 
tune.search(X_train, y_train, epochs = 200, batch_size = 20, validation_data = (X_test, y_test), callbacks = [LR_decay, Early_stop])

LSTM_model = tune.get_best_models()[0]
LSTM_model.save('/content/Git-6300/models/LSTM_model1.h5')
hyperparameters = tune.get_best_hyperparameters()[0]
print(hyperparameters.values)

Trial 50 Complete [00h 00m 30s]
val_loss: 0.002046700567007065

Best val_loss So Far: 0.0009332177578471601
Total elapsed time: 00h 29m 19s
{'n_hidden': 0, 'neurons_visible': 78, 'activate1': 'sigmoid', 'dropout1': 0.7, 'neurons_hidden1': 20, 'dropout2': 0.7, 'neurons_hidden2': 50, 'dropout3': 0.7, 'neurons_hidden3': 50}


## Configure the Prophet Model

In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import ParameterGrid
!pip install yfinance
import yfinance as yf
import datetime as dt

#Generate basic financial data from yfinance for prophet training
def prophet_data(ticker, start_day, month, year):
    delta = dt.timedelta(days = 100)
    data = yf.download(ticker, (dt.date(year, month, start_day) - delta)).reset_index()
    print(data.head())
    data = data.rename(columns = {'Date':'ds', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Adj Close': 'y', 'Volume':'volume'})
    print(data.head())
    data = data.loc[:,['ds', 'y']]
    return data

MSFT_data = prophet_data('MSFT',1, 1,2016)
GOOG_data = prophet_data('GOOG', 1, 1, 2016)
# This function is used to get the train data and test data
def data_preparation(df):
  data= df.loc[:,["ds","y"]]
  return data
  
# Define parameter grid to search over
param_grid = {
    'seasonality_mode': ['additive', 'multiplicative'],
    'changepoint_prior_scale': [0.01, 0.1, 1.0],
    'seasonality_prior_scale': [0.01, 0.1, 1.0],
}

# Initialize minimum error and best parameters
min_error = float('inf')
best_params = {}

# Loop through all parameter combinations
for params in ParameterGrid(param_grid):
    print('Testing parameters:', params)

    # Initialize Prophet model with specified hyperparameters
    model = Prophet(**params)
    model.fit(data_preparation(MSFT_data))

    # Perform time series cross-validation
    df_cv = cross_validation(model=model, initial='1000 days', horizon='10 days', period='10 days')

    # Calculate performance metrics
    df_metrics = performance_metrics(df_cv)

    # Calculate mean cross-validation error
    mean_cv_error = df_metrics['mse'].mean()

    # Update minimum error and best parameters if new minimum is found
    if mean_cv_error < min_error:
        min_error = mean_cv_error
        best_params = params

# Print best hyperparameters and corresponding error
print('Best parameters:', best_params)
print('Minimum cross-validation error:', min_error)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed
        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-09-23  43.930000  44.169998  43.509998  43.869999  38.970509  17145200
1 2015-09-24  43.450001  44.130001  43.270000  43.910000  39.006042  27905600
2 2015-09-25  44.480000  44.730000  43.759998  43.939999  39.032688  29384600
3 2015-09-28  43.830002  44.090000  43.209999  43.290001  38.455288  27613800
4 2015-09-29  43.369999  43.570000  43.049999  43.439999  38.588531  32763600
          ds       open       high        low      close          y    volume
0 2015-09-23  43.930000  44.169998  43.509998  43.869999  38.970509  17145200
1 2015-09-24  43.450001  44.130001  43.270000  43.910000  39.006042  27905600
2 2015-09-25  44.480000  44.730000  43.759998  43.939999  39.032688  29384600
3 2015-09-28  43.830002  44.090000  43.209999  43.29

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



        Date       Open       High        Low      Close  Adj Close    Volume
0 2015-09-23  31.102501  31.446501  31.000000  31.118000  31.118000  29418000
1 2015-09-24  30.832001  31.365999  30.620001  31.290001  31.290001  44802000
2 2015-09-25  31.488501  31.488501  30.549999  30.598499  30.598499  43480000
3 2015-09-28  30.517000  30.730249  29.469000  29.744499  29.744499  62554000
4 2015-09-29  29.864000  30.250000  29.511000  29.748501  29.748501  46190000
          ds       open       high        low      close          y    volume
0 2015-09-23  31.102501  31.446501  31.000000  31.118000  31.118000  29418000
1 2015-09-24  30.832001  31.365999  30.620001  31.290001  31.290001  44802000
2 2015-09-25  31.488501  31.488501  30.549999  30.598499  30.598499  43480000
3 2015-09-28  30.517000  30.730249  29.469000  29.744499  29.744499  62554000
4 2015-09-29  29.864000  30.250000  29.511000  29.748501  29.748501  46190000
Testing parameters: {'changepoint_prior_scale': 0.01, 'seasonal

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/ixqsggkh.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/nt9ligxn.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=36449', 'data', 'file=/tmp/tmp5i36n3bq/ixqsggkh.json', 'init=/tmp/tmp5i36n3bq/nt9ligxn.json', 'output', 'file=/tmp/tmp5i36n3bq/prophet_modelvdn56h88/prophet_model-20230328003054.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
00:30:54 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:30:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 174 forecasts with cutoffs between 2018-06-21 00:00:00 and 2023-03-17 00:00:00


  0%|          | 0/174 [00:00<?, ?it/s]

...

Testing parameters: {'changepoint_prior_scale': 1.0, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 1.0}


01:15:59 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Making 174 forecasts with cutoffs between 2018-06-21 00:00:00 and 2023-03-17 00:00:00


  0%|          | 0/174 [00:00<?, ?it/s]

DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/4i3_bz35.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/9fvstaxq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.9/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=67647', 'data', 'file=/tmp/tmp5i36n3bq/4i3_bz35.json', 'init=/tmp/tmp5i36n3bq/9fvstaxq.json', 'output', 'file=/tmp/tmp5i36n3bq/prophet_modelgh0a2y7x/prophet_model-20230328011559.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:15:59 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:16:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/2i9l6r2f.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp5i36n3bq/ost88e_q.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/l

Best parameters: {'changepoint_prior_scale': 1.0, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 0.01}
Minimum cross-validation error: 375.6045357845492


## Generate predictions on test data

In [ ]:
# We use the best parameter to fit the model
def get_data(df):
  Best_parameters={'changepoint_prior_scale': 0.1, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 1.0}
  # We use the best parameter to fit the model
  final_model = Prophet(**Best_parameters)
  final_model.fit(data_preparation(df))
  df = cross_validation(model=final_model, initial='100 days', horizon='10 days', period='10 days')
  return df

#generate msft and google prediction datasets 
GOOG_data = get_data(GOOG_data)[['ds', 'yhat']]
MSFT_full = get_data(MSFT_data)[['ds', 'yhat']]
MSFT_prof_train = MSFT_full.set_index('ds').iloc[:1613]
MSFT_prof_test = MSFT_full.set_index('ds').iloc[1613:]

## Create Dataset to train hybrid model

In [ ]:
from keras.saving.legacy.save import load_model
#function to scale prophet predictions before passing to hybrid model
def prophet_scaler(prophet_df):
  prophet_scaler = MinMaxScaler()
  prophet_df['yhat'] = prophet_scaler.fit_transform(prophet_df[['yhat']])
  return prophet_df['yhat']

#create training data using predictions from lstm and prophet
lstm = load_model('/content/Git-6300/models/LSTM_model1.h5')
def return_hybrid_data(lstm_model, prophet_data, LSTM_data, LSTM_target):
  preds_dict = {}
  preds_dict['lstm_pred'] = lstm_model.predict(LSTM_data).flatten()
  print(lstm_model.predict(LSTM_data).shape)
  preds_dict['prophet_pred_microsoft'] = np.array(prophet_scaler(MSFT_prof_train).reset_index()['yhat'])
  hybrid_train = pd.DataFrame(preds_dict)
  hybrid_train, target = lstm_split(hybrid_train, y_train, len(hybrid_train))
  return hybrid_train, target

hybrid_train, target = return_hybrid_data(lstm, MSFT_prof_train, X_train, y_train)

OSError: ignored

## Configure Hybrid Model

In [ ]:
LR = 0.01
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.metrics import Precision
from keras.optimizers import Adam

#General LSTM model based on randomsearch results
def multiple_models(x_train, y_train):
    hybrid = Sequential()
    hybrid.add(LSTM(units = 58, activation = "sigmoid", input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = False))
    hybrid.add(Dropout(0.7))
    hybrid.add(Dense(units =1))
    hybrid.compile(loss = 'mean_squared_error', optimizer = Adam(lr = LR), metrics = ["mean_squared_error"])
    Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                              patience=25, verbose=1, mode='auto',
                              baseline=0, restore_best_weights=True)
    hybrid.fit(x_train, y_train, epochs = 200, batch_size = 20, callbacks = [Early_stop])
    return model
    
#train the hybrid model
hybrid1 = multiple_models(hybrid_train, y_train[10:])
hybrid1.save('/content/Git-6300/models/hybrid1.h5')

NameError: ignored

# Test Single Company Models (LSTM vs. Hybrid)

In [ ]:
#Configure LSTM test using google data and micosoft validation data
goog_test, goog_target = lstm_split(test_data, test_target, len(test_data))
LSTM_predictions = lstm.predict(goog_test)
LSTM_validation = lstm.predict(X_test)
abs = scaler_list[1].inverse_transform(LSTM_validation) - scaler_list[1].inverse_transform(y_test)

#Generate google test data for hybrid model
google_ft, google_target = return_hybrid_data(lstm, GOOG_data, test_data, test_target)
hybrid_val_feat, hybrid_val_target = return_hybrid_data(lstm, MSFT_prof_test, X_test, y_test)
prophet_predictions = prophet_scaler(GOOG_data)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot MAE and loss
#plt.figure(figsize=(10, 6))
plt.plot(hybrid_history.history['mae'], label='MAE')
plt.plot(hybrid_history.history['loss'], label='loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
def compute_metrics(true_series, forecast):
    """Helper to print MSE and MAE"""
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

In [ ]:
mse, mae = compute_metrics(y1, predictions)
print(mae)

In [ ]:
plt.plot(y1, label="actual")
plt.plot(predictions, label="predicted")
plt.xlabel("Timesteps")
plt.ylabel("Value")
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.legend()
plt.show()

# Expanded Model

In [ ]:
LR = 0.01
def multiple_models(x_train, y_train):
    hybrid = Sequential()
    hybrid.add(LSTM(units = 58, activation = "sigmoid", input_shape = (x_train.shape[1], x_train.shape[2]), return_sequences = False))
    hybrid.add(Dropout(0.7))
    hybrid.add(Dense(units =1))
    hybrid.compile(loss = 'mean_absolute_error', optimizer = Adam(lr = LR), metrics = ["mean_squared_error"])
    Early_stop = EarlyStopping(monitor='loss', min_delta=0, 
                              patience=25, verbose=1, mode='auto',
                              baseline=0, restore_best_weights=True)
    hybrid.fit(x_train, y_train, epochs = 200, batch_size = 20, callbacks = [Early_stop])
    return hybrid
    
def create_data_lists(list_of_feats, list_of_targets):
  training_list = []
  target_list = []
  for data, target in zip(list_of_feats, list_of_targets):
    data_x, data_y = lstm_split(data, target, len(data))
    print(data_x.shape)
    training_list.append(data_x)
    target_list.append(data_y)
  return training_list, target_list

#Train each of the lstm models on training companies
path = '/content/Git-6300/expanded_data/'
ticker_list = ["MSFT", "NFLX", "AMZN", "TSLA"]
hybrid_scalers, targets, frames = scale(path)
training_list, target_list = create_data_lists(frames, targets)

for tick, x_train, y_train in zip(ticker_list, training_list, target_list):
  model = multiple_models(x_train, y_train)
  model.save('/content/Git-6300/ex_models/'+tick+'_lstm.h5')

## Train Expanded Hybrid Model Using Predictions of all Sub-Models

In [ ]:
#Create Train Data for Hybrid Model Using Microsoft Predictions
from keras.saving.legacy.save import load_model
import os
def hybrid_trainer(direct, ticker_list, hybrid_train):
    preds_dict = {}
    path = direct
    for num, model in enumerate(os.listdir(path)):
        model = load_model(path+model)
        prediction = model.predict(hybrid_train).flatten()
        print(prediction.shape)
        preds_dict[ticker_list[num]] = prediction
    preds_df = pd.DataFrame(preds_dict)
    return preds_df

#generate training data for expanded hybrid model
hybrid_training_data =  hybrid_trainer('/content/Git-6300/ex_models/', ticker_list, X_train) 
print(hybrid_training_data.shape, MSFT_prof_train)
hybrid_training_data = pd.concat([hybrid_training_data, MSFT_prof_train], axis = 1)

hybrid_training_data, y_train = lstm_split(hybrid_training_data.drop('index', axis = 1), y_train, len(hybrid_training_data))
print(type(hybrid_training_data))
print(hybrid_training_data.shape)

In [ ]:
#Create and train the expanded hybrid model
hybrid_expanded = multiple_models(hybrid_training_data, y_train)
hybrid_expanded.save('/content/Git-6300/models/hybrid_expanded.h5')

## Test the Expanded Hybrid Model on Microsoft and Goole Test Data


In [ ]:
#Configure hybrid model testing data based on predictions from each of the four underlying models and prophet
from os import listdir
google_test = {}
microsoft_test = {}
path = '/content/Git-6300/ex_models/'
for num, model in enumerate(os.listdir(path)):
  model = load_model(model)
  goog_preds = model.predict(google_ft)
  micro_preds = model.predict(X_test)
  google_test[ticker_list[num]] = goog_preds
  microsoft_test[ticker_list[num]] = micro_preds
google_test = pd.DataFrame(google_test)
microsoft_test = pd.DataFrame(microsoft_test)
google_test['prophet'] = GOOG_data
microsoft_test['prophet'] = MSFT_prof_test
google_test = lstm_split(google_test, google_target, len(google_test))
microsoft_test = lstm_split(microsoft_test, y_test, len(microsoft_test))


In [ ]:
#Predict google and microsoft test data using the trained hybrid model
hybrid_expanded = load_model('/content/Git-6300/models/hybrid_expanded.h5')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot MAE and loss
#plt.figure(figsize=(10, 6))
plt.plot(hybrid_history.history['mae'], label='MAE')
plt.plot(hybrid_history.history['loss'], label='loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

def compute_metrics(true_series, forecast):
    """Helper to print MSE and MAE"""
    mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
    mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

    return mse, mae

mse, mae = compute_metrics(y1, predictions)
print(mae)
plt.plot(y1, label="actual")
plt.plot(predictions, label="predicted")
plt.xlabel("Timesteps")
plt.ylabel("Value")
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.legend()
plt.show()